In [1]:
'''
11월 2일 수요일
'''

# 필요한 라이브러리 호출
from requests import Session, get
from datetime import datetime
from urllib import parse
from bs4 import BeautifulSoup
from html import escape, unescape
import json
import re

# 신규 세션 생성
session = Session()

In [2]:
def get_login_cookie(facebookID,facebookPW):
    '''
    get_login_cookie(session,facebookID,facebookPW) -> session, login_headers, dtsg, resp
    페이스북 아이디, 페이스북 패스워드를 넣으면 해당 계정으로 로그인한 세션, 쿠키를 포함한 헤더, dtsg, 로그인 결과를 반환한다.
    '''
    
    session = Session()
    
    # 첫 요청에 들어가는 헤더 생성 (Edge 브라우저 기준 기본값)
    initHeaders=dict()
    h='''accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: html
accept-language: ko
sec-ch-ua: "Microsoft Edge";v="107", "Chromium";v="107", "Not=A?Brand";v="24"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: none
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'''

    # 줄마다 구분 한 것을 ': ' 기준으로 나눠서 앞의 것을 키, 뒤의 것을 값으로 initHeaders 딕셔너리에 입력 
    for line in h.split('\n'):
        res = line.split(': ')
        initHeaders[res[0]]=res[1]

    # 필요한 값들을 얻기 위해 페이스북 초기화면으로 이동
    initURL = 'http://m.facebook.com/'
    init_resp = session.get(initURL, headers=initHeaders)
    
    # 페이스북 초기 화면에서 받을 수 있는 값들을 찾는 과정
    
    # action="/login/device-based/regular/login/?refsrc=deprecated&amp;lwv=100&amp;refid=8", 비밀번호를 잊으셨나요? 사이의 HTML 문서 추출
    loginHTML = init_resp.text
    cropstart='action="/login/device-based/regular/login/?refsrc=deprecated&amp;lwv=100&amp;refid=8"'
    cropend="비밀번호를 잊으셨나요?"
    paramHTML = loginHTML[loginHTML.index(cropstart):loginHTML.index(cropend)]
    
    # <input type="hidden" 로 시작해서 > 로 끝나는 구간을 찾아낸다. (HTML 상에 존재하는 페이로드에 넣을 입력값이 있는 곳을 찾음)
    startpoint = '<input type="hidden"'
    endpoint = ">"
    prev_end=0

    payloadFromHTML=dict() # 결과를 저장할 딕셔너리
    
    
    while True:
        try: # startpoint, endpoint가 있는지 확인
            start = paramHTML.index(startpoint,prev_end)
            end = paramHTML.index(endpoint,start)
        except: # startpoint, endpoint가 해당 구간내에 존재하지 않는다면 while 루프를 탈출한다.
            break
        
        # startpoint와 endpoint 사이의 문자열을 저장한다.
        KVstring = paramHTML[start:end]

        # 해당 문자열 내의 "value"가 있는지 확인한다. (value가 없는 경우도 있으며 이런 경우를 무시하기 위한 과정)
        if "value" in KVstring:
            kv = re.findall('.+name="([^"]+)".+value="([^"]+)"',KVstring) # 정규식으로 키와 값 쌍을 얻어낸다.
            payloadFromHTML[kv[0][0]] = kv[0][1] # 정규식으로 얻어낸 키-값 쌍을 저장한다.
        else: # value가 없는 경우 그대로 다음 문자열로 이행한다.
            pass
        
        # endpoint를 기준으로 이후의 새로운 <input 절을 찾는다.
        prev_end=end
        
    
    for key in payloadFromHTML.keys():
        print("%s : %s"%(key,payloadFromHTML[key]))
    
    #  페이로드에 이메일과 평문 비밀번호를 넣는다.
    payloadFromHTML['email']=facebookID
    payloadFromHTML['encpass'] = '#PWD_BROWSER:0:{}:{}'.format(payloadFromHTML['m_ts'],facebookPW)
    
    # (필요 여부 불명) 하드코딩 된 헤더값 추가
    headers=initHeaders.copy()
    headers['x-requested-with']='XMLHttpRequest'
    headers['x-response-format']='JSONStream'
    
    # 로그인 서비스에 지금까지의 값들을 입력한다.
    loginURL = 'https://m.facebook.com/login/device-based/login/async/?refsrc=deprecated&lwv=100'
    resp = session.post(loginURL, data=payloadFromHTML, headers=headers)
    
    # (필요 여부 불명) 로그인 성공 후 세션에 등록된 쿠키를 문자열로 처리해서 다시 헤더에 넣기 위해 저장
    loginCookie = session.cookies
    
    loginHeaders = dict()
    cookieString=""
    for key in ['datr','sb','fr','c_user', 'xs']:
        cookieString+="%s=%s; "%(key,loginCookie[key]) # 문자열에 '키=값; '형식으로 값을 이어붙인다.
    
    cookieString=cookieString[:-2] # 모든 키에 대해서 작업 수행 후 마지막의 '; '를 제거하고 저장한다.
    loginHeaders['cookie']=cookieString # 해당 쿠키문자열을 헤더에 추가한다.
    
    '''
    # 결과로 반환된 응답의 dtsg 값을 찾는다. (fb_dtsg에 들어갈값 으로 추측됨)
    rerespjs = json.loads(resp.text[9:]) # 9개의 문자열을 뺀 뒷부분은 json으로 변환 가능하니 json으로 변환한다.
    jstext = rerespjs['payload']['actions'][1]['code'] # dtsg 가 포함된 문자열
    start = jstext.index('dtsg') # dtsg의 내용
    end = jstext.index('expire',start) # dtsg가 나온 이후 처음으로 expire가 언급된 구간을 찾는다.
    dtsg = re.findall('"token":"([^"]+)"',jstext[start:end])[0] # 해당 구간에서 토큰값을 저장한다.
    
    dtsgSep = dtsg.split(':')
    dtsgSep[1] = "49"
    dtsgSep[2] = payloadFromHTML['m_ts']
    
    dtsgCal = ""
    for factor in dtsgSep:
        dtsgCal+=(factor+":")
    dtsgCal=dtsgCal[:-1]
    '''
    
    lgnResp = session.get("https://m.facebook.com/login/save-device/?login_source=login",
                          headers=initHeaders,
                          data={"login_source":"login"})
    lgnHTML = lgnResp.text
    linestart = lgnHTML.index("MPageLoadClientMetrics.init(")
    start = lgnHTML.index('"',linestart)
    end = lgnHTML.index('"',start+1)
    dtsg = lgnHTML[start+1:end]
    
    return loginHeaders, dtsg, resp # 쿠키를 문자열로 반환, dtsg, resp (디버그용) 을 반환한다.

In [3]:
headers, dtsg, result_resp = get_login_cookie("seungirumd+1@gmail.com","pfbmw246")

lsd : AVoTCCGmlUE
jazoest : 2922
m_ts : 1667375942
li : RiNiY2aUiRBeLXK8EP8niMKt
try_number : 0
unrecognized_tries : 0
had_cp_prefilled : false
had_password_prefilled : false
is_smart_lock : false
bi_xrwh : 0
_fb_noscript : true


In [4]:
dtsg

'NAcOtfcp3es9oLWtMrHqru5lyQn8jP9Vt8K-G_cVaZSUiAkNjiNOmdA:40:1667375942'

In [5]:
headers

{'cookie': 'datr=RiNiYyKKB4OgPyfR2RMC8abo; sb=RiNiY3SAWfptWATpXOwmieGw; fr=0fBWaTtkF3vB5L7QS.AWU2t7AiFHgV9Ufcz5JKc8d_4Tw.BjYiNG.ab.AAA.0.0.BjYiNG.AWXcBAy11fk; c_user=100087280119490; xs=40%3ABO8dIXu5lbOtYw%3A2%3A1667375942%3A-1%3A-1'}

In [6]:
session = Session()

In [7]:
# 냉동만두에 대한 게시글을 가져오기 위해 필요한 패러미터들
data = {'fb_dtsg': dtsg,
        '__a': '1'}
url = 'https://m.facebook.com/search/posts/'
get_params = {'q': '냉동만두', 'source': 'filter', 'isTrending': '0', 'cursor' : '',  'pn' : '1', 'tsid': ''}

full_url = url+'?'
keys = list(get_params.keys())
values = list(get_params.values())

for i in range(len(keys)):
    full_url += '%s=%s&'%(keys[i],values[i]) # 서비스 URL+? 이후로 검색 패러미터의 '키:값&'형식으로 모든 키-값 쌍을 추가한다.
full_url = full_url[:-1] # 마지막 &를 제거하고 저장

In [8]:
'''
오류가 발생하기 시작하는 부분

내일 (11/02/수) 확인해야 하는 내용

cookie에 더 들어가는 m_pixel, locale 등의 값들이 필수인지, 필수라면 어디서 구하는지 확인
페이로드에 들어가는 __a (비로그인시 1인 것으로 보임), dtsg의 2,3 구간의 값들을 어떻게 채우는지 확인 (dtsg는 긴 값:짧은 값:짧은 값 형식)
'''
#요청보내기
resp = session.post(full_url, data=data, headers=headers)

#JSON형식으로 변환 후 html 코드 (우리가 원하는 텍스트 정보) 추출
rstext= resp.text[9:]
rsjson = json.loads(rstext)
#해당 페이지의 텍스트 정보를 포함하고 있는 html코드, BeautifulSoup로 파싱해서 정규식 추출해야함

unescape(rsjson['payload']['actions'][0]['html'])


'<div class="_45fx _4d7h" id="BrowseResultsContainer"><div><div id="u_cl5tba_0_Lj"><div><div class="_10y_ _10z6"><div class="_7k7 storyStream"><div class="_1p82" id="u_cl5tba_2_pu"><div class="_2h4o" id="u_cl5tba_5_MV"><div><div class="_18qg _3u_7"><div class="_55wo _56bf _5rgl" id="u_cl5tba_3_SX"><div class="story_body_container"><div class="_581t"><table class="_4g33"><tbody><tr><td class="_5s61"><div class="_67lm"></div></td><td class="_4g34 _52we"><div><h3 class="_52ja _52jh _4vc- _lqt"><span><strong><a href="/pig123/?__tn__=C&paipv=0&eav=AfZA8sD9GTZePxcRvSZHBY6EliBlpr5SfLeXnNB8wjOQB5VPNlu2b40YNC_igYgrpe8">돼지야 살쪘다고 징징대지를 말든가 먹지를 말든가 둘중에 하나만해라</a></strong></span></h3></div></td></tr></tbody></table></div><div class="_5rgn" style="" data-ft="{"tn":"*s"}"><div><span><p>가정교육이 참 중요하다라는걸 또 깨닫는다</p></span></div></div><div class="_5rgo _27x0" data-ft="{"tn":"H"}"><a class="_5tip _5tfq" href="http://lm.facebook.com/l.php?u=http%3A%2F%2Fhot-post.com%2F%3Fp%3D119984%26fbclid%3DIwAR3Gr4E0sxknh

In [9]:
#다음 커서 찾기, 정규식 이용
p = re.compile('cursor=([a-zA-Z0-9_-]*[^&])')
next_cur = p.search(unescape(rsjson['payload']['actions'][0]['html'])).group(1)

#url에 들어가는 파라미터 중 'cursor'만 교체
get_params['cursor'] = next_cur
full_url = url+'?'
keys = list(get_params.keys())
values = list(get_params.values())
for i in range(len(keys)):
    full_url += '%s=%s&'%(keys[i],values[i]) 
full_url = full_url[:-1]